In [73]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm
import autocorrect
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [74]:
data_path = './data/Tweets.csv'
data = pd.read_csv(data_path)

In [76]:
data = data[data['airline_sentiment'] != 'neutral']

# | class balancing
data['airline_sentiment'].value_counts().plot.bar()
nr_positive = data[data['airline_sentiment'] == 'positive'].shape[0]
g = data.groupby('airline_sentiment')
data = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)).reset_index(drop=True)
data['airline_sentiment'].value_counts()

positive    2363
negative    2363
Name: airline_sentiment, dtype: int64

In [77]:
data['labels'] = data['airline_sentiment'].astype('category').cat.codes

In [13]:
def remove_punctuation(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_punctuation_two(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def process_word(word, spellcheck=False, stemming=False, lemmatizing=True):
    if spellcheck:
        word = autocorrect.spell(word)
    if stemming:
        stemmer = nltk.stem.snowball.SnowballStemmer('english')
        word = stemmer.stem(word)
    if lemmatizing:
        lemmatizer = nltk.stem.WordNetLemmatizer()
        lemmatizer.lemmatize(word)

def process_text(text):
    # TODO: expand contractions
    text = remove_punctuation(text.lower())
    text = nltk.tokenize.word_tokenize(text)
    return [process_word(word) for word in text if word not in nltk.corpus.stopwords.words('english')]

tqdm.pandas()
data['text_processed'] = data['text'].progress_apply(process_text)


 21%|████████████████▎                                                            | 1005/4726 [00:07<00:27, 133.09it/s]


 43%|████████████████████████████████▉                                            | 2024/4726 [00:14<00:18, 147.46it/s]


 74%|█████████████████████████████████████████████████████████▏                   | 3510/4726 [00:21<00:05, 225.04it/s]


100%|█████████████████████████████████████████████████████████████████████████████| 4726/4726 [00:27<00:00, 171.81it/s]

In [78]:
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    tokenizer=TreebankWordTokenizer().tokenize,
    #token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2), # include 1-grams and 2-grams
    min_df=2, # only keep terms that appear in at least 2 documents
    max_features=10000)

x = word_vectorizer.fit_transform(data['text'])
y = data['labels']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

# cross validation
model = LogisticRegression(C=0.1, solver='sag')
np.mean(cross_val_score(model, x_train, y_train, cv=3, scoring='roc_auc'))

In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Gridsearch/CV with preprocessing
pipe = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier' , LogisticRegression())])

param_grid = [
    {'classifier__penalty' : ['l1'],
    'classifier__C' : np.logspace(-4, 4, 10),
    'classifier__solver' : ['liblinear', 'saga'],
     
    'vectorizer__sublinear_tf' : [True, False],
    'vectorizer__ngram_range' : [(1, 1), (1, 2), (1, 3)],
    'vectorizer__min_df' : [1, 2, 3],
    'vectorizer__strip_accents' : ['unicode'],
    'vectorizer__max_features' : [5000, 10000],
    'vectorizer__stop_words' : [None, 'english']}
]

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 3, verbose=True, scoring='roc_auc', n_jobs=-1)
best_clf = clf.fit(data['text'], data['labels'])

print('best score: {}\nbest params: {}'.format(best_clf.best_score_, best_clf.best_params_))  

Fitting 3 folds for each of 1440 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2438 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2988 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 3638 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed: 26.6min finished
c:\users\nicolas\workspace\environments\nlp\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


best score: 0.9555474147595265
best params: {'classifier__C': 1291.5496650148827, 'classifier__penalty': 'l1', 'classifier__solver': 'saga', 'vectorizer__max_features': 10000, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': None, 'vectorizer__strip_accents': 'unicode', 'vectorizer__sublinear_tf': True}


In [142]:
results = pd.DataFrame(best_clf.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)

results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__penalty,param_classifier__solver,param_vectorizer__max_features,param_vectorizer__min_df,param_vectorizer__ngram_range,param_vectorizer__stop_words,param_vectorizer__strip_accents,param_vectorizer__sublinear_tf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1264,14.609591,0.037692,0.057181,0.000470,1291.55,l1,saga,10000,1,"(1, 2)",None,unicode,True,"{'classifier__C': 1291.5496650148827, 'classif...",0.961312,0.951922,0.953406,0.955547,0.004122,1
1408,16.786709,0.326396,0.068483,0.007739,10000,l1,saga,10000,1,"(1, 2)",None,unicode,True,"{'classifier__C': 10000.0, 'classifier__penalt...",0.960715,0.952157,0.953330,0.955401,0.003789,2
1409,16.358228,0.018779,0.056848,0.000815,10000,l1,saga,10000,1,"(1, 2)",None,unicode,False,"{'classifier__C': 10000.0, 'classifier__penalt...",0.960628,0.951649,0.953451,0.955243,0.003879,3
1265,14.732841,0.152434,0.057847,0.003259,1291.55,l1,saga,10000,1,"(1, 2)",None,unicode,False,"{'classifier__C': 1291.5496650148827, 'classif...",0.960550,0.951736,0.953046,0.955112,0.003884,4
1120,9.641878,0.039545,0.063164,0.003672,166.81,l1,saga,10000,1,"(1, 2)",None,unicode,True,"{'classifier__C': 166.81005372000558, 'classif...",0.960739,0.950541,0.953293,0.954858,0.004308,5


In [159]:
data['pred_sentiment'] = data['text'].apply(lambda row: best_clf.best_estimator_.predict([row])[0])

data[['labels', 'text', 'pred_sentiment']]


,labels,text,pred_sentiment
0,0,@AmericanAir my flight was Cancelled Flightled...,0
1,0,@USAirways Been on hold for over 5 hours! I be...,0
2,0,@americanair never fails to disappoint. waitin...,0
3,0,@USAirways bad weather shouldn't mean bad service,0
4,0,@USAirways what an amazing day. Delayed 4 hrs ...,0
5,0,@JetBlue should've been on one of your flights...,0
6,0,@JetBlue our flight out no Tv. Now our flight ...,0
7,0,nope. RT @JetBlue: Our fleet's on fleek. http:...,0
8,0,@AmericanAir Really?They closed the gate at 10...,0
9,0,.@AmericanAir: the best the agent could do is ...,0
